In [35]:
import pandas as pd
import numpy as np

from collections import Counter
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,cohen_kappa_score, matthews_corrcoef, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from matplotlib import rcParams

pd.options.mode.chained_assignment = None 

In [36]:
def train_test_data_split(raw_data, train_mapping, test_mapping):
    train_data = pd.DataFrame(columns=raw_data.columns)
    test_data = pd.DataFrame(columns=raw_data.columns)
    
    for case_id in raw_data.index:
        case_general_id = case_id[0:20]
        if case_general_id in train_mapping.values[:,1]:
            train_data.loc[case_id]= raw_data.loc[case_id, :]
        elif case_general_id in test_mapping.values[:,1]:
            test_data.loc[case_id]= raw_data.loc[case_id, :]
            
    return train_data, test_data

In [37]:
def split_modalities(input_data):
    
    flair= pd.DataFrame(columns=input_data.columns)
    t1c = pd.DataFrame(columns=input_data.columns)
    t1= pd.DataFrame(columns=input_data.columns)
    t2 = pd.DataFrame(columns=input_data.columns)
    
    for case_id in input_data.index:
        if "FLAIR" in case_id:
            flair.loc[case_id]= input_data.loc[case_id, :]
        elif "T1C" in case_id:
            t1c.loc[case_id]= input_data.loc[case_id, :]
        elif "T1" in case_id:
            t1.loc[case_id]= input_data.loc[case_id, :]
        elif "T2" in case_id:
            t2.loc[case_id]= input_data.loc[case_id, :]
            
            
    labels = (flair.loc[:, 'Grade']).to_numpy()
   
    flair.drop('Grade',axis='columns', inplace=True)
    t1c.drop('Grade',axis='columns', inplace=True)
    t1.drop('Grade',axis='columns', inplace=True)
    t2.drop('Grade',axis='columns', inplace=True)
    
    return flair, t1c, t1, t2, labels

In [38]:
def prepare_scaler(dataset):
    dataset_values = dataset.loc[:, dataset.columns != "Grade"]
    standard_scaler = StandardScaler().fit(dataset_values)
    return standard_scaler

In [39]:
def apply_pca(dataset, scaler):
    dataset_values = dataset.loc[:, dataset.columns != "Grade"]
    dataset_labels = dataset.loc[:, "Grade"]
    dataset_index = dataset.index
        
    dataset_values[dataset_values.columns] = scaler.transform(dataset_values[dataset_values.columns])
        
    pca = PCA(.95)
    
    pca.fit(dataset_values)
        
    dataset_values = pca.transform(dataset_values)
    
    dataset_pca = pd.DataFrame(dataset_values, index=dataset_index)
    dataset_pca["Grade"] = dataset_labels
    
    return dataset_pca, pca

In [40]:
def equalize_mapping(mapping):
    is_HGG =  mapping['Grade']=='HGG'
    is_LGG =  mapping['Grade']=='LGG'
    hggs = mapping[is_HGG]
    lggs = mapping[is_LGG]
    
    x = len(lggs)
    hggs = hggs.sample(n=x)
    equalized_mapping = pd.concat([hggs, lggs])
    return equalized_mapping

In [41]:
def equalize_dataset(mapping, dataset):
    mapping_ids = mapping["ID"].tolist()
    dataset.index = dataset.index.astype('str')
    equalized_train_dataset = pd.DataFrame()
    for id in mapping_ids:   
        case_df = dataset[dataset.index.str.contains(id)]
        equalized_train_dataset = pd.concat([equalized_train_dataset, case_df])
    return equalized_train_dataset

In [42]:
def preprocess_test_dataset(dataset, scaler, pca):
    dataset_values = dataset.loc[:, dataset.columns != "Grade"]
    dataset_labels = dataset.loc[:, "Grade"]
    dataset_index = dataset.index
    
    dataset_values[dataset_values.columns] = scaler.transform(dataset_values[dataset_values.columns])
        
    dataset_values = pca.transform(dataset_values)
    
    preprocessed_dataset = pd.DataFrame(dataset_values, index=dataset_index)
    preprocessed_dataset["Grade"] = dataset_labels
    
    return preprocessed_dataset

In [43]:
def check_majority_proba(proba_arr):
    
    hgg_proba = 0
    lgg_proba = 0
    
    for modality_proba in proba_arr:
        hgg_proba += modality_proba[0]
        lgg_proba += modality_proba[1]
        
    if  hgg_proba >= lgg_proba:
        return "HGG"
    else:
        return "LGG"

In [44]:
def prepare_metrics(overall_acc, overall_kappa, overall_mcc, overall_precision, overall_recall, overall_dice, overall_hausdorff, overall_volume): 
    general_accuracy = pd.DataFrame({'Accuracy': overall_acc})
    general_kappa = pd.DataFrame({'Kappa': overall_kappa})
    general_mcc = pd.DataFrame({'MCC': overall_mcc})
    general_precision = pd.DataFrame({'Precision': overall_precision})
    general_recall = pd.DataFrame({'Recall': overall_recall})
    general_dice = pd.DataFrame({'Dice': overall_dice})
    general_hausdorff = pd.DataFrame({'Hausdorff': overall_hausdorff})
    general_volume = pd.DataFrame({'Volume': overall_volume})
        
    accuracy_mean = float(general_accuracy.mean())
    accuracy_median = float(general_accuracy.median())
    accuracy_Q1 = general_accuracy.Accuracy.quantile([0.25]).to_numpy()[0]
    accuracy_Q3 = general_accuracy.Accuracy.quantile([0.75]).to_numpy()[0]
    accuracy_IQR = accuracy_Q3 - accuracy_Q1
    kappa_mean = float(general_kappa.mean())
    kappa_median = float(general_kappa.median())
    kappa_Q1 = general_kappa.Kappa.quantile([0.25]).to_numpy()[0]
    kappa_Q3 = general_kappa.Kappa.quantile([0.75]).to_numpy()[0]
    kappa_IQR = kappa_Q3 - kappa_Q1
    mcc_mean = float(general_mcc.mean())
    mcc_median = float(general_mcc.median())
    mcc_Q1 = general_mcc.MCC.quantile([0.25]).to_numpy()[0]
    mcc_Q3 = general_mcc.MCC.quantile([0.75]).to_numpy()[0]
    mcc_IQR = mcc_Q3 - mcc_Q1
    precision_mean = float(general_precision.mean())
    precision_median = float(general_precision.median())
    precision_Q1 = general_precision.Precision.quantile([0.25]).to_numpy()[0]
    precision_Q3 = general_precision.Precision.quantile([0.75]).to_numpy()[0]
    precision_IQR = precision_Q3 - precision_Q1
    recall_mean = float(general_recall.mean())
    recall_median = float(general_recall.median())
    recall_Q1 = general_recall.Recall.quantile([0.25]).to_numpy()[0]
    recall_Q3 = general_recall.Recall.quantile([0.75]).to_numpy()[0]
    recall_IQR = recall_Q3 - recall_Q1
    dice_mean = float(general_dice.mean())
    dice_median = float(general_dice.median())
    dice_Q1 = general_dice.Dice.quantile([0.25]).to_numpy()[0]
    dice_Q3 = general_dice.Dice.quantile([0.75]).to_numpy()[0]
    dice_IQR = dice_Q3 - dice_Q1
    hausdorff_mean = float(general_hausdorff.mean())
    hausdorff_median = float(general_hausdorff.median())
    hausdorff_Q1 = general_hausdorff.Hausdorff.quantile([0.25]).to_numpy()[0]
    hausdorff_Q3 = general_hausdorff.Hausdorff.quantile([0.75]).to_numpy()[0]
    hausdorff_IQR = hausdorff_Q3 - hausdorff_Q1
    voulme_mean = float(general_volume.mean())
    volume_median = float(general_volume.median())
    volume_Q1 = general_volume.Volume.quantile([0.25]).to_numpy()[0]
    volume_Q3 = general_volume.Volume.quantile([0.75]).to_numpy()[0]
    volume_IQR = 0 
    
    accuracy_list = [accuracy_mean, accuracy_median, accuracy_Q1, accuracy_Q3, accuracy_IQR]
    kappa_list = [kappa_mean, kappa_median, kappa_Q1, kappa_Q3, kappa_IQR]
    mcc_list = [mcc_mean, mcc_median, mcc_Q1, mcc_Q3, mcc_IQR]
    precision_list = [precision_mean, precision_median, precision_Q1, precision_Q3, precision_IQR]
    recall_list = [recall_mean, recall_median, recall_Q1, recall_Q3, recall_IQR]
    dice_list = [dice_mean, dice_median, dice_Q1, dice_Q3, dice_IQR]
    hausdorff_list = [hausdorff_mean, hausdorff_median, hausdorff_Q1, hausdorff_Q3, hausdorff_IQR]
    volume_list = [voulme_mean, volume_median, volume_Q1, volume_Q3, volume_IQR]

    rows_labels = ["Mean", "Median", "Q1", "Q3", "IQR"]
    column_labels = ["Accuracy", "Kappa", "MCC", "Precision","Recall", "Dice", "Hausdorff", "Volume"]
    metrics = pd.DataFrame(index=rows_labels, columns=column_labels)

    metrics["Accuracy"] = accuracy_list
    metrics["Kappa"] = kappa_list
    metrics["MCC"] = mcc_list
    metrics["Precision"] = precision_list
    metrics["Recall"] = recall_list
    metrics["Dice"] = dice_list
    metrics["Hausdorff"] = hausdorff_list
    metrics["Volume"] = volume_list

    rounded_metrics = metrics.round(3)
    rounded_metrics.to_csv("general_metrics.csv")
    
    return rounded_metrics

In [49]:
# dataset = pd.read_csv("output/2D_mask_features.csv", index_col=0)
# dataset = pd.read_csv("output/3D_mask_features.csv", index_col=0)
# dataset = pd.read_csv("output/ENS_mask_features.csv", index_col=0)
dataset = pd.read_csv("output/GT_mask_features.csv", index_col=0)

name_mapping = pd.read_csv("mapping/name_mapping.csv")

fold_iterator = 1

skf = StratifiedKFold(n_splits=5)

y = name_mapping.values[:,0]
X = name_mapping.values[:,1]
strat_label = name_mapping.values[:,8]

dice = name_mapping.values[:,2]
hausdorff = name_mapping.values[:,3]
volume = name_mapping.values[:, 9]

overall_acc = []
overall_kappa = []
overall_mcc = []
overall_prc = []
overall_rec = []
overall_dice = []
overall_hausdorff = []
overall_volume = []

column_labels = ["Fold", "Accuracy", "Kappa", "MCC", "Precision", "Recall", "Dice", "Hausdorff", "Volume"]
fold_metrics = pd.DataFrame(columns=column_labels)

columns_dc = ["ID", "GT", "PRED"]
detailed_classification = pd.DataFrame(columns = columns_dc)


for train_index, test_index in skf.split(X, strat_label):
    print("\n")
    print("FOLD NUMBER: ", fold_iterator)
    print("\n")

    scaler = None
    pca = None

    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    strat_label_train, strat_label_test = strat_label[train_index], strat_label[test_index]
    
    dice_train, dice_test = dice[train_index], dice[test_index]
    hausdorff_train, hausdorff_test = hausdorff[train_index], hausdorff[test_index]
    volume_train, volume_test =  volume[train_index], volume[test_index]
    
    train_mapping = pd.DataFrame({'Grade': y_train, 'ID': X_train, 'Strat_Label': strat_label_train , 'Dice': dice_train, 'Hausdorff': hausdorff_train, 'Volume': volume_train})   
    test_mapping = pd.DataFrame({'Grade': y_test, 'ID': X_test, 'Strat_Label': strat_label_test, 'Dice': dice_test, 'Hausdorff': hausdorff_test, 'Volume': volume_test})

    equalized_train_mapping = equalize_mapping(train_mapping)

    (train_dataset, test_dataset) = train_test_data_split(dataset, train_mapping, test_mapping)

    scaler = prepare_scaler(train_dataset)

    pca_train_dataset, pca = apply_pca(train_dataset, scaler)

    equalized_train_dataset = equalize_dataset(equalized_train_mapping, pca_train_dataset)
    
    (flair_data, t1c_data, t1_data, t2_data, labels) = split_modalities(equalized_train_dataset)
    
    # flair_classifier = LogisticRegression()
    # t1c_classifier = LogisticRegression()
    # t1_classifier = LogisticRegression()
    # t2_classifier = LogisticRegression()

    # flair_classifier = KNeighborsClassifier()
    # t1c_classifier = KNeighborsClassifier()
    # t1_classifier = KNeighborsClassifier()
    # t2_classifier = KNeighborsClassifier()
    
    # flair_classifier = DecisionTreeClassifier()
    # t1c_classifier = DecisionTreeClassifier()
    # t1_classifier = DecisionTreeClassifier()
    # t2_classifier = DecisionTreeClassifier()
    
    # flair_classifier = SVC(gamma='auto', probability=True)
    # t1c_classifier = SVC(gamma='auto', probability=True)
    # t1_classifier = SVC(gamma='auto', probability=True)
    # t2_classifier = SVC(gamma='auto', probability=True)
    
    # flair_classifier = RandomForestClassifier()
    # t1c_classifier = RandomForestClassifier()
    # t1_classifier = RandomForestClassifier()
    # t2_classifier = RandomForestClassifier()
    
    flair_classifier = ExtraTreesClassifier()
    t1c_classifier = ExtraTreesClassifier()
    t1_classifier = ExtraTreesClassifier()
    t2_classifier = ExtraTreesClassifier()
    
    flair_classifier.fit(flair_data.to_numpy(), labels)
    t1c_classifier.fit(t1c_data.to_numpy(), labels)
    t1_classifier.fit(t1_data.to_numpy(), labels)
    t2_classifier.fit(t2_data.to_numpy(), labels)

    preprocessed_test_dataset = preprocess_test_dataset(test_dataset, scaler, pca)

    (test_flair_data, test_t1c_data, test_t1_data, test_t2_data, test_labels) = split_modalities(preprocessed_test_dataset)
    
    number_of_test_cases = len(test_mapping)

    test_results = []

    for i in range(number_of_test_cases):
        probability_array = []

        flair_result = flair_classifier.predict_proba(test_flair_data.iloc[i].to_numpy().reshape(1,-1))[0]
        probability_array.append(flair_result)

        t1c_result = t1c_classifier.predict_proba(test_t1c_data.iloc[i].to_numpy().reshape(1,-1))[0]
        probability_array.append(t1c_result)

        t1_result = t1_classifier.predict_proba(test_t1_data.iloc[i].to_numpy().reshape(1,-1))[0]
        probability_array.append(t1_result)

        t2_result = t2_classifier.predict_proba(test_t2_data.iloc[i].to_numpy().reshape(1,-1))[0]
        probability_array.append(t2_result)

        final_prediction = check_majority_proba(probability_array)

        test_results.append(final_prediction)

    test_results = np.array(test_results)

    test_cases_id = test_mapping["ID"].to_numpy()
    for i in range(len(test_results)):
        detailed_classification = detailed_classification.append({"ID": test_cases_id[i], "GT": test_labels[i], "PRED": test_results[i]}, ignore_index=True)

    
    clsr = classification_report(test_labels, test_results, output_dict=True)
    print(confusion_matrix(test_labels, test_results))
    print(classification_report(test_labels, test_results))

    
    acc = accuracy_score(test_labels, test_results)
    kappa = cohen_kappa_score(test_labels, test_results)
    mcc = matthews_corrcoef(test_labels, test_results)
    precision = clsr["HGG"]["precision"]
    recall = clsr["LGG"]["recall"]
    fold_dice = float(test_mapping.Dice.mean())
    fold_hausdorff = float(test_mapping.Hausdorff.mean())
    fold_volume = float(test_mapping.Volume.mean())

    fold_name = "Fold_" + str(fold_iterator)
    fold_metrics = fold_metrics.append({"Fold": fold_name, "Accuracy": round(acc, 3), "Kappa": round(kappa, 3),
                                        "MCC": round(mcc, 3),"Precision": round(precision, 3), "Recall": round(recall, 3),
                                        "Dice": round(fold_dice, 3), "Hausdorff": round(fold_hausdorff, 3), "Volume": round(fold_volume, 3)}, ignore_index=True)
    
    fold_metrics.to_csv("fold_metrics.csv", index=False)
    
    overall_acc.append(acc)
    overall_kappa.append(kappa)
    overall_mcc.append(mcc)
    overall_prc.append(precision)
    overall_rec.append(recall)
    overall_dice.append(fold_dice)
    overall_hausdorff.append(fold_hausdorff)
    overall_volume.append(fold_volume)

    fold_iterator += 1
    
detailed_classification.to_csv("detailed_metrics.csv", index=False)
general_metrics = prepare_metrics(overall_acc, overall_kappa, overall_mcc, overall_prc, overall_rec, overall_dice, overall_hausdorff, overall_volume)



FOLD NUMBER:  1


[[59  0]
 [ 2 13]]
              precision    recall  f1-score   support

         HGG       0.97      1.00      0.98        59
         LGG       1.00      0.87      0.93        15

    accuracy                           0.97        74
   macro avg       0.98      0.93      0.96        74
weighted avg       0.97      0.97      0.97        74



FOLD NUMBER:  2


[[52  7]
 [ 7  8]]
              precision    recall  f1-score   support

         HGG       0.88      0.88      0.88        59
         LGG       0.53      0.53      0.53        15

    accuracy                           0.81        74
   macro avg       0.71      0.71      0.71        74
weighted avg       0.81      0.81      0.81        74



FOLD NUMBER:  3


[[43 15]
 [ 3 13]]
              precision    recall  f1-score   support

         HGG       0.93      0.74      0.83        58
         LGG       0.46      0.81      0.59        16

    accuracy                           0.76        74
   macro av